In [122]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
#from keras.models import load_model
import urllib.request
from urllib3 import PoolManager
import re

In [123]:
#Gjenstår: Hent odds

## Find odds for 7 days at a time.

### Source 1: soccerbase

In [124]:
import gzip
import urllib.request
games = []

In [125]:

# url_string2 = None
# with urllib.request.urlopen('http://www.soccerbase.com/matches/home.sd?type=2') as response:
#     gzipFile = gzip.GzipFile(fileobj=response)
#     url_string2 = gzipFile.read().decode("UTF-8")
#     url_string2 = re.search('(?<=soccer.gamelist.Timekeeper.addGames\({).*?(?=soccer.gamelist)', url_string2)[0]


In [126]:
# matches = re.finditer('(?<=\=)(.*?)\_V\_(.*?)\=(.*?)\"', url_string2)
# for match in matches:
#     games.append([match[3],re.sub("_", " ", match[1]).lower(),re.sub("_", " ", match[2]).lower()])

### Source 2: supersport

In [127]:
# response = urllib.request.urlopen('https://www.supersport.com/football/fixtures')
# url_string = response.read()
# url_string = url_string.decode("UTF-8")
# url_string = re.search('(?<=FIXTURES)(.|\s)*', url_string)[0]

In [128]:
# dates = re.finditer("(?<=Header\">)(.*?)(?=class=\"table)", url_string)

# for day in dates:
    
#     date = re.search('.*?(?=</td>)',day[0])[0]
#     matches = re.finditer("(?<=>)(..:..)(.*?)(?=</td><td class)",day[0])
    
#     for match in matches:
#         time_of_match = match[0][:5]
#         teams_re = re.finditer("(?<=st\'>)(.*?)(?=</td>)", match[0])
#         game = [date]
#         for team in teams_re:
            
#             game.append(team[0].lower())
#         games.append(game)

### Source 3: thefishy

In [129]:
url_string3 = None
with urllib.request.urlopen('https://thefishy.co.uk/nextweeksfixtures.php') as response:
    url_string3 = response.read().decode("UTF-8") + " 12/12</th>"

In [130]:
dates = re.finditer("(\ ../..<(.|\s)*?)(?=\ \w\w/\w\w</th>)", url_string3)

for date in dates:
    day = date[0][1:6] + "/18"
    matches = re.finditer("(?<=\'>)(.*?)\ v\ (.*?)(?=<)", date[0])
    for match in matches:
        away_team = match[2]
        home_team = re.search('(?<=team2)(.*?>)(.*?)(?=$)',match[1])[2]
        
#         print(home_team, away_team)
        
        games.append([day, home_team.lower(), away_team.lower()])


In [131]:
%run update_stash.py

unzipping  ../../Football_Data/Zipstash\data_0.zip
unzipping  ../../Football_Data/Zipstash\data_1.zip
unzipping  ../../Football_Data/Zipstash\data_10.zip
unzipping  ../../Football_Data/Zipstash\data_11.zip
unzipping  ../../Football_Data/Zipstash\data_12.zip
unzipping  ../../Football_Data/Zipstash\data_13.zip
unzipping  ../../Football_Data/Zipstash\data_14.zip
unzipping  ../../Football_Data/Zipstash\data_15.zip
unzipping  ../../Football_Data/Zipstash\data_16.zip
unzipping  ../../Football_Data/Zipstash\data_17.zip
unzipping  ../../Football_Data/Zipstash\data_18.zip
unzipping  ../../Football_Data/Zipstash\data_19.zip
unzipping  ../../Football_Data/Zipstash\data_2.zip
unzipping  ../../Football_Data/Zipstash\data_20.zip
unzipping  ../../Football_Data/Zipstash\data_21.zip
unzipping  ../../Football_Data/Zipstash\data_22.zip
unzipping  ../../Football_Data/Zipstash\data_23.zip
unzipping  ../../Football_Data/Zipstash\data_24.zip
unzipping  ../../Football_Data/Zipstash\data_25.zip
unzipping  ../.

In [132]:
all_seasons = []
for filename in os.listdir('C:/Users/bfesc/Documents/Football_Data/Stash/'):
    #if filename[0] == 'B':
    with open("C:/Users/bfesc/Documents/Football_Data/Stash/" + filename) as file:
        season = file.read().rstrip().split('\n')
        for i in range(len(season)):
            season[i] = season[i].rstrip(',').split(',')
            for j in range(len(season[i])):
                season[i][j] = season[i][j].lower()
        all_seasons.append(season)

In [133]:
columns = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'WHH', 'WHD', 'WHA']
columns = [col.lower() for col in columns]
all_data = pd.DataFrame(columns=columns)
for season in all_seasons:
    if not np.in1d(columns, season[0]).all():
        continue
    if len(season[0]) != max([len(s) for s in season]):
        for i in range(len(season)):
            season[i] = season[i][:len(season[0])]
    df = pd.DataFrame(season[1:])
    df.columns = season[0]
    all_data = all_data.append(df[columns])
    
all_data['date'] = pd.to_datetime(all_data['date'], dayfirst=True)
all_data.index = all_data['date']
all_data = all_data.drop(columns=['date','div'])
for column in all_data.columns:
    all_data = all_data[pd.notnull(all_data[column])]
all_data = all_data[pd.notnull(all_data.index)]
all_data = all_data.iloc[np.where(all_data['whh'] != "")]
all_data = all_data.iloc[np.where(all_data['whd'] != "")]
all_data = all_data.iloc[np.where(all_data['wha'] != "")]
all_data = all_data.iloc[np.where(all_data['fthg'] != "")]
all_data = all_data.sort_index(ascending=False)
all_data['ftr'].iloc[np.where(all_data['ftr'] == 'a')] = 0
all_data['ftr'].iloc[np.where(all_data['ftr'] == 'd')] = 1
all_data['ftr'].iloc[np.where(all_data['ftr'] == 'h')] = 2
all_data['fthg'] = all_data['fthg'].astype(int)
all_data['ftag'] = all_data['ftag'].astype(int)
all_data['whh'] = all_data['whh'].astype(np.float32)
all_data['whd'] = all_data['whd'].astype(np.float32)
all_data['wha'] = all_data['wha'].astype(np.float32)

In [134]:
#from difflib import SequenceMatcher
from fuzzywuzzy import process

def unknown_team(home_team, away_team, team_dict, unknown = "away"):

    if unknown == "away":
        team_scores = []
        teams = team_dict[home_team]['awayteam'].unique() #Away or HomeTeam doesnt really matter 
        
        team_scores = process.extract(away_team,teams)
        #for team in teams:
        #    team_scores.append(SequenceMatcher(None,team,away_team).ratio())
        
        if team_scores[0][1] < 80:
            return None
        
        #team_scores = np.argsort(team_scores)
        
        away_team = team_scores[0][0]
    else:
        teams = team_dict[away_team]['hometeam']
        team_scores = []
            
        team_scores = process.extract(home_team,teams)
        
        if team_scores[0][1] < 80:
            return None
        
        #team_scores = np.argsort(team_scores)
        
        home_team = team_scores[0][0]
        
    return home_team, away_team


In [135]:
def make_team_dicts(data):
    
    ht = data["hometeam"].str.lower()
    at = data["awayteam"].str.lower()
    
    all_teams = np.unique(np.append(ht, at))
    team_dict = {}
    
    for team in all_teams:
        team_dict[team] = data.iloc[np.where(np.logical_or(ht == team, 
                                                      at == team))].sort_index(ascending=False)
        
    return team_dict

In [136]:
team_dict = make_team_dicts(all_data)

In [137]:
SequenceMatcher(None,"dortmund","bayerrr").ratio()

0.13333333333333333

In [138]:
from fuzzywuzzy import process

In [139]:
# Sanity check
unknown_team("dortmund", "bayern", team_dict)

('dortmund', 'bayern munich')

In [140]:
def make_live_data(live_data, team_dict):


    training_data = pd.DataFrame(columns=['Date','HomeTeam', 'AwayTeam', 'DaysSinceLastHTMatch', 'DaysSinceLastATMatch', 'HomeTeamForm', 'AwayTeamForm',
                                          'HomeTeamHomeForm','AwayTeamAwayForm', 'HomeTeamOffense', 'HomeTeamDefense',
                                          'AwayTeamOffense', 'AwayTeamDefense', 'LastInterTeamGame', 
                                           'Result'])


    history = pd.Timedelta('90d')
    year = pd.Timedelta('365d')
    for i in tqdm(range(len(live_data))):#
        game = live_data[i]
        home_team = game[1]
        away_team = game[2]
        game_date = pd.to_datetime(game[0]) 

        if home_team not in team_dict.keys() and away_team not in team_dict.keys():
            continue
        elif home_team not in team_dict.keys():
            try:
                home_team, away_team = unknown_team(home_team, away_team, team_dict, unknown="home")
            except:
                print(f"fant ikke {home_team}")
                continue
        elif away_team not in team_dict.keys():
            try:
                home_team, away_team = unknown_team(home_team, away_team, team_dict, unknown="away")
            except:
                print(f"fant ikke {away_team}")
                continue
        ht_games = team_dict[home_team].loc[game_date - pd.Timedelta('1h'):game_date - year]
        at_games = team_dict[away_team].loc[game_date - pd.Timedelta('1h'):game_date - year]

        #result = game['FTR']
        result = 0

        ht_hgames = ht_games.iloc[np.where(ht_games['hometeam'] == home_team)]
        ht_agames = ht_games.iloc[np.where(ht_games['awayteam'] == home_team)]

        at_hgames = at_games.iloc[np.where(at_games['hometeam'] == away_team)]
        at_agames = at_games.iloc[np.where(at_games['awayteam'] == away_team)]


        if len(ht_hgames) > 1 and len(at_agames) > 1:

            ht_time_diff = np.log((game_date - ht_games.index).days.values + 1)
            at_time_diff = np.log((game_date - at_games.index).days.values + 1)

            ht_h_td = ht_time_diff[np.where(ht_games['hometeam'] == home_team)]
            ht_a_td = ht_time_diff[np.where(ht_games['awayteam'] == home_team)]
            at_h_td = at_time_diff[np.where(at_games['hometeam'] == away_team)]
            at_a_td = at_time_diff[np.where(at_games['awayteam'] == away_team)]


            it_games = np.append(np.where(ht_hgames['awayteam'] == away_team)[0],
                                 np.where(ht_agames['hometeam'] == away_team)[0])

            litgr = 0
            if len(it_games) > 0:
                litgr += np.mean((ht_games['fthg'].iloc[it_games].values - ht_games['ftag'].iloc[it_games].values) / ht_time_diff[it_games])

            dslhtm = ht_time_diff[0]
            dslatm = at_time_diff[0]

            ht_form = np.mean(np.append(((ht_hgames['fthg'].values - ht_hgames['ftag'].values) / ht_h_td),
                              (ht_agames['ftag'].values - ht_agames['fthg'].values) / ht_a_td))
            at_form = np.mean(np.append((at_hgames['fthg'].values - at_hgames['ftag'].values) / at_h_td,
                              (at_agames['ftag'].values - at_agames['fthg'].values) / at_a_td))

            ht_hform = np.mean((ht_hgames['fthg'].values - ht_hgames['ftag'].values) / ht_h_td)
            at_aform = np.mean((at_agames['ftag'].values - at_agames['fthg'].values) / at_a_td)

            hto = np.mean(np.append(ht_hgames['fthg'].values / ht_h_td, ht_agames['ftag'].values / ht_a_td))
            htd = -np.mean(np.append(ht_hgames['ftag'].values / ht_h_td, ht_agames['fthg'].values /ht_a_td))

            ato = np.mean(np.append(at_hgames['fthg'] / at_h_td, at_agames['ftag'].values / at_a_td))
            atd = -np.mean(np.append(at_hgames['ftag'].values / at_h_td, at_agames['fthg'].values / at_a_td))

            
            training_data = training_data.append(pd.DataFrame([[game_date,home_team, away_team, dslhtm, dslatm, ht_form, at_form, ht_hform, at_aform,
                                                                hto, htd, ato, atd, litgr, result]], columns=training_data.columns))

    return training_data
    
    

In [141]:
training_data = make_live_data(games,team_dict)

  2%|█▋                                                                               | 18/844 [00:02<02:57,  4.66it/s]

fant ikke celta vigo b
fant ikke zamora


  4%|███                                                                              | 32/844 [00:03<01:48,  7.51it/s]

fant ikke cd ebro
fant ikke ud ibiza-eivissa
fant ikke fc andorra


  4%|███▍                                                                             | 36/844 [00:03<01:22,  9.84it/s]

...

fant ikke meppen


 76%|█████████████████████████████████████████████████████████████                   | 644/844 [00:27<00:06, 29.28it/s]

fant ikke pau
fant ikke villefranche
fant ikke concarneau


100%|████████████████████████████████████████████████████████████████████████████████| 844/844 [00:28<00:00, 30.12it/s]

fant ikke vizela
fant ikke fatih karagumruk
fant ikke club atlas


In [142]:
training_data.shape

(237, 15)

In [143]:
teams_playing = np.unique(np.array(games)[:,1].tolist() + np.array(games)[:,2].tolist())

In [144]:
def get_odds_betfair():
    html = PoolManager()
    webpage = html.request("GET","https://www.betfair.com/sport/football")

    webpage_l = webpage.data.decode("UTF-8").lower()

    end_cue = "</span>\n</div>\n</a>\n</div>\n</div>\n</div>\n</li>"
    start_cue = '<div class="avb-col avb-col-markets">'

    wp_all_games = re.findall(f"{start_cue}.*?{end_cue}",webpage_l,flags=re.DOTALL)

    game_odds = []

    for wp in wp_all_games:
        try:
            odds = [float(o) for o in re.findall("(?<=\n)\d{1,2}\.\d{1,2}(?=\n)",wp)[2:]]
            hometeam, awayteam = re.findall('<span class="team-name" title="(.*?)">',wp)

            game_odds.append({"hometeam" : hometeam,
                                "awayteam" : awayteam,
                                "homeodds" : odds[0],
                                "awayodds" : odds[-1]})
        except Exception as e:
            print(e)

    return game_odds

def kelly_criterion(prob_win, net_fractional_odds):
    prob_loss = 1 - prob_win
    
    return (((net_fractional_odds*prob_win) - prob_loss) / prob_loss)

In [145]:
odds = pd.DataFrame(get_odds_betfair())

list index out of range


In [146]:
odds = odds.loc[~((odds.homeodds > 3) & (odds.awayodds > 3))]

In [147]:
#inntil videre så settes treff på minst ett lag som krav.
print(odds.loc[odds.hometeam.isin(teams_playing) | odds.awayteam.isin(teams_playing)].shape)
data_with_odds = training_data.merge(odds,left_on="HomeTeam",right_on="hometeam",how="inner")
data_with_odds = data_with_odds.append(training_data.merge(odds,left_on="AwayTeam",right_on="awayteam",how="inner"))
data_with_odds.shape[0]

(32, 4)


7

In [148]:
from xgboost import XGBClassifier
gb = XGBClassifier()
gb.load_model("model_2021.json")

In [149]:
data_with_odds.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'DaysSinceLastHTMatch',
       'DaysSinceLastATMatch', 'HomeTeamForm', 'AwayTeamForm',
       'HomeTeamHomeForm', 'AwayTeamAwayForm', 'HomeTeamOffense',
       'HomeTeamDefense', 'AwayTeamOffense', 'AwayTeamDefense',
       'LastInterTeamGame', 'Result', 'hometeam', 'awayteam', 'homeodds',
       'awayodds'],
      dtype='object')

In [150]:
data_with_odds = data_with_odds.rename(columns={"homeodds":"HomeTeamOdds","awayodds":"AwayTeamOdds"}).drop_duplicates(subset=["HomeTeam","AwayTeam"])

In [151]:
data_with_odds.shape

(6, 19)

In [154]:
data_with_odds.head(1)

,Date,HomeTeam,AwayTeam,DaysSinceLastHTMatch,DaysSinceLastATMatch,HomeTeamForm,AwayTeamForm,HomeTeamHomeForm,AwayTeamAwayForm,HomeTeamOffense,HomeTeamDefense,AwayTeamOffense,AwayTeamDefense,LastInterTeamGame,Result,hometeam,awayteam,HomeTeamOdds,AwayTeamOdds
0,2018-04-18,rotherham,birmingham,0.693147,1.386294,0.110711,-0.157403,0.181479,-0.274192,0.371245,-0.260534,0.179934,-0.337336,0,0,rotherham,birmingham,2.7,2.5


In [186]:
#data_with_odds = data_with_odds.assign(LastInterTeamGame=lambda df: df.LastInterTeamGame.astype(float))

In [210]:
pred_cols = ['DaysSinceLastHTMatch', 'DaysSinceLastATMatch', 'HomeTeamForm', 'AwayTeamForm',
      'HomeTeamHomeForm','AwayTeamAwayForm', 'HomeTeamOffense', 'HomeTeamDefense',
      'AwayTeamOffense', 'AwayTeamDefense', 'LastInterTeamGame', 
      'HomeTeamOdds', 'AwayTeamOdds']

livepreds = gb.predict(data_with_odds[pred_cols])
probs = gb.predict_proba(data_with_odds[pred_cols])#[range(len(livepreds)),livepreds.tolist()]
odds = data_with_odds[["AwayTeamOdds","Result","HomeTeamOdds"]].values[range(len(livepreds)),livepreds.tolist()]

In [196]:
BALANCE=933

def bet_size(cash, n_bets, win_probs, odds):
    return (cash*kelly_criterion(win_probs,odds-1)) / n_bets


for i in range(len(livepreds)):
    print(f"bet_size = {bet_size(BALANCE,len(livepreds),probs[i],odds[i])}")
    print(f"{data_with_odds.iloc[i].Date}: {data_with_odds.iloc[i].HomeTeam} - {data_with_odds.iloc[i].AwayTeam} ")
    print(["Away","Draw","Home"][livepreds[i]])
    print()

bet_size = -14.851893508062444
2018-04-18 00:00:00: rotherham - birmingham 
Away

bet_size = -13.459549270035192
2018-04-18 00:00:00: rotherham - middlesbrough 
Away

bet_size = -14.851893508062444
2018-04-18 00:00:00: derby - birmingham 
Away

bet_size = -31.203108501030602
2018-04-18 00:00:00: cadiz - cordoba 
Away

bet_size = -33.21925261758334
2018-04-18 00:00:00: milan - genoa 
Home

bet_size = -19.56980053386873
2018-04-18 00:00:00: paris sg - st etienne 
Home



In [207]:
#Det som må gjøres:
#1) Hent uavgjort-data
#2) Sjekk kelly-score for odds/probs for hver av utfallene
#3) Hvis ett eller flere av utfallene har positiv kelly-score, anbefal det største!

In [212]:
probs

array([[0.37616694, 0.3002782 , 0.32355487],
       [0.3784814 , 0.2988397 , 0.3226789 ],
       [0.37616694, 0.3002782 , 0.32355487],
       [0.42085052, 0.31897032, 0.2601792 ],
       [0.11306633, 0.22409625, 0.66283745],
       [0.0827442 , 0.13266684, 0.784589  ]], dtype=float32)

In [217]:
all_odds = data_with_odds[["HomeTeamOdds","AwayTeamOdds"]].values
for i in range(len(livepreds)):
    print("Kelly home score : {:.2f}".format(kelly_criterion(probs[i,-1],all_odds[i,0]-1)))
    print("Kelly away score : {:.2f}".format(kelly_criterion(probs[i,0],all_odds[i,1]-1)))
    print()

Kelly home score : -0.19
Kelly away score : -0.10

Kelly home score : -0.19
Kelly away score : -0.09

Kelly home score : -0.19
Kelly away score : -0.10

Kelly home score : -0.12
Kelly away score : -0.20

Kelly home score : -0.21
Kelly away score : -0.24

Kelly home score : -0.13
Kelly away score : -0.19



In [198]:
probs[i], odds[i], kelly_criterion(probs[i],odds[i]-1)

(0.784589, 1.24, -0.12585080729176032)

In [193]:
kelly_criterion(np.mean(probs),np.mean(odds)-1)

#bet_size(BALANCE,len(livepreds),probs,odds)

0.03937087973442492

In [189]:
161*6

966

In [ ]:
everton - tottenham 
blackburn - derby 
reading - cardiff 
crawley town - cheltenham 
st mirren - inverness c 
motherwell - morton 
girona - zaragoza 
leipzig - hoffenheim 
spal - ascoli 
lille - montpellier 
boavista - pacos ferreira 
antalyaspor - rizespor 
forfar - clyde 
preston - derby 
brentford - cardiff 
peterboro - northampton 
stevenage - cheltenham 
darmstadt - greuther furth 
bourg peronnas - orleans 
roda - nac breda 

In [207]:
livepreds[i]

array([0.46973372, 0.269813  , 0.26045328], dtype=float32)